In [66]:
import random

# --- Datos del problema ---
tasks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
precedence = {
    'C': ['A'],
    'D': ['A'],
    'E': ['B'],
    'F': ['C', 'D'],
    'G': ['E', 'F'],
    'H': ['G'],
    'I': ['F'],
    'J': ['H', 'I']
}

def is_valid(order):
    position = {task: i for i, task in enumerate(order)}
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                return False
    return True

def fitness(order):
    position = {task: i for i, task in enumerate(order)}
    score = 100
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                score -= 30
            else:
                # Penalizar distancia grande entre pred y task
                score -= (position[task] - position[pred] - 1) * 2
    return score


## Generar población

In [67]:
def create_individual():
    return random.sample(tasks, len(tasks))

def create_population(size=20):
    return [create_individual() for _ in range(size)]

population = create_population()
for ind in population:
    print(ind, "-> Fitness:", fitness(ind))


['E', 'J', 'G', 'B', 'H', 'I', 'C', 'D', 'A', 'F'] -> Fitness: -120
['J', 'B', 'G', 'H', 'I', 'D', 'A', 'C', 'F', 'E'] -> Fitness: -98
['J', 'F', 'C', 'E', 'H', 'G', 'I', 'A', 'D', 'B'] -> Fitness: -126
['C', 'D', 'F', 'I', 'G', 'E', 'B', 'J', 'A', 'H'] -> Fitness: -68
['A', 'H', 'D', 'J', 'I', 'F', 'B', 'E', 'G', 'C'] -> Fitness: -48
['F', 'J', 'C', 'B', 'A', 'E', 'D', 'G', 'I', 'H'] -> Fitness: -84
['F', 'A', 'I', 'D', 'H', 'B', 'J', 'C', 'G', 'E'] -> Fitness: -62
['G', 'C', 'I', 'E', 'F', 'D', 'A', 'B', 'J', 'H'] -> Fitness: -170
['B', 'G', 'A', 'F', 'H', 'C', 'I', 'E', 'J', 'D'] -> Fitness: -64
['G', 'H', 'D', 'I', 'C', 'J', 'A', 'E', 'F', 'B'] -> Fitness: -104
['G', 'C', 'A', 'F', 'B', 'E', 'D', 'J', 'I', 'H'] -> Fitness: -112
['F', 'G', 'E', 'C', 'H', 'A', 'B', 'I', 'D', 'J'] -> Fitness: -80
['G', 'A', 'B', 'J', 'C', 'E', 'D', 'F', 'H', 'I'] -> Fitness: -56
['H', 'B', 'A', 'E', 'J', 'F', 'D', 'C', 'G', 'I'] -> Fitness: -60
['D', 'F', 'I', 'G', 'E', 'C', 'B', 'H', 'J', 'A'] -> Fit

## Operadores GA: selección, cruce, mutación

In [68]:
def selection(pop):
    a, b = random.sample(pop, 2)
    return a if fitness(a) > fitness(b) else b

def crossover(p1, p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    fill = [x for x in p2 if x not in child]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

def mutate(ind, prob=0.2):
    if random.random() < prob:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


## Evolución

In [69]:
def evolve(population, generations=50):
    for gen in range(generations):
        new_pop = []
        for _ in range(len(population)):
            p1 = selection(population)
            p2 = selection(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
        best = max(population, key=fitness)
        print(f"Gen {gen+1:2d}: {best} | Fitness: {fitness(best)} | Válido: {is_valid(best)}")
    return best

best = evolve(population)
print("\nMejor solución encontrada:", best)


Gen  1: ['I', 'A', 'C', 'D', 'H', 'F', 'B', 'E', 'J', 'G'] | Fitness: 4 | Válido: False
Gen  2: ['I', 'A', 'C', 'F', 'D', 'B', 'E', 'G', 'H', 'J'] | Fitness: 14 | Válido: False
Gen  3: ['H', 'A', 'C', 'D', 'J', 'F', 'B', 'E', 'I', 'G'] | Fitness: 14 | Válido: False
Gen  4: ['H', 'A', 'C', 'D', 'J', 'F', 'I', 'B', 'E', 'G'] | Fitness: 20 | Válido: False
Gen  5: ['H', 'A', 'C', 'D', 'J', 'F', 'I', 'B', 'E', 'G'] | Fitness: 20 | Válido: False
Gen  6: ['A', 'H', 'D', 'B', 'E', 'C', 'F', 'I', 'J', 'G'] | Fitness: 30 | Válido: False
Gen  7: ['A', 'H', 'D', 'B', 'E', 'C', 'F', 'G', 'I', 'J'] | Fitness: 34 | Válido: False
Gen  8: ['A', 'C', 'H', 'D', 'B', 'E', 'F', 'I', 'J', 'G'] | Fitness: 34 | Válido: False
Gen  9: ['A', 'C', 'H', 'D', 'B', 'E', 'F', 'G', 'I', 'J'] | Fitness: 38 | Válido: False
Gen 10: ['A', 'C', 'H', 'D', 'B', 'E', 'F', 'G', 'I', 'J'] | Fitness: 38 | Válido: False
Gen 11: ['A', 'C', 'D', 'B', 'E', 'H', 'F', 'G', 'I', 'J'] | Fitness: 42 | Válido: False
Gen 12: ['A', 'C', 'D'